# NAIP Image Chip Creation
### Earthshot Institute Riparian Ecosystems Lab

This script uses the EI_Riparian module to run a process that develops a set of 256x256 image chips from NAIP imagery that are used to train a computer vision model to identify more Beaver Dams. 

This script relies on a set beaver dam shapefiles from Dr. Emily Fairfax's research on Beaver Dams. 

In [4]:
import EI_Riparian as EIR
import rasterio, glob, warnings, os
import geopandas as gpd

In [ ]:
# Get your Current Working Directory. Should be the location of this file. 
# If you would like to identify a new location for Data downlaod and creation, do so in the cell below.  

os.getcwd()

In [17]:
# This is your opportunity to select a different location for downloading the DEMs and NAIP image chips. 
# You might do this if you don't have much space on your machine and wish to use an external drive.
# However, please keep in mind that you will need to copy over the BEAVER_DAM_SHAPEFILES 
# folder to that location for the script to run properly.  

DataFolder = 'DATA'       # Copy the new path here.  
EIR.change_dir(DataFolder)

# Please only run this cell once.  

In [ ]:
%%time

n_counter = 1

## Input your own new file path - outside of the repo. This downloads a LOT of files.  
for InputData in glob.glob('BEAVER_DAM_SHAPEFILES'):
    print(f"Working on location {n_counter} of {len(glob.glob('BEAVER_DAM_SHAPEFILES/*.shp'))}...")

    warnings.filterwarnings("ignore")  ## optional. I'm getting an number of depreciated warnings... 
    rasterList = EIR.Download_DEM(InputData)
    
    print ("Download Complete")

    rasterList = EIR.deleteOldFiles(os.path.dirname(rasterList[0]))  
    
    
    for eachDEM in rasterList:

        
        DEM_path = EIR.crop_byBoundary(eachDEM, InputData)
        
        
        if DEM_path is not None:
            
        
            DEM_name = os.path.split(eachDEM)[-1].split('.')[0]
            DEM = rasterio.open(DEM_path)

            inputName = os.path.split(InputData)[-1].split('.')[0]

            dirList = [f'NAIP/CHIPS_{inputName}']
            for x in dirList:
                if not os.path.isdir(x):
                    os.mkdir(x)

            streamNet = EIR.watershed_preProcessing(DEM_path)
            print ("Watershed pre-Processing Complete")


            points, points_path = EIR.makePointsFromLines(streamNet, 100)
            print ("Points Creation Complete")

            ## Set up the yes/no dam question for training data
            buff = gpd.read_file(points_path).buffer(128, cap_style=3)
            buff.to_file(f'POINTS/{inputName}_tempOUT.shp')
            buff_gdf = gpd.read_file(f'POINTS/{inputName}_tempOUT.shp')

            dams = gpd.read_file(InputData)
            damsRepro = dams.to_crs(buff.crs)
            dams_union = damsRepro.geometry.unary_union


            print (f"Now, creating NAIP chips for {inputName}, yes beaver dam intersect.")
            
            try:
                ## Yes, there is an intersection between image chip and beaver dam, output images
                yes_dams = buff_gdf[buff_gdf.geometry.intersects(dams_union)]
                yes_path = f'POINTS/{inputName}_yes_dams.shp'
                yes_dams.to_file(yes_path)
                yes_gdf = gpd.read_file(yes_path)
                yes_repro = yes_gdf.to_crs(DEM.crs)
                NAIP_Chips_folder = EIR.points_toNAIP_Chips(yes_repro, inputName, 'yes')
            except:
                print ("No (yes) NAIP imagery found")
                pass
            
            
            try:
                print (f"Now, creating NAIP chips for {inputName}, no beaver dam intersect.")
                ## No, there is an intersection between image chip and beaver dam, output images
                no_dams = buff_gdf[~buff_gdf.geometry.intersects(dams_union)]
                no_path = f'POINTS/{inputName}_no_dams.shp'
                no_dams.to_file(no_path)
                no_gdf = gpd.read_file(no_path)
                no_repro = no_gdf.to_crs(DEM.crs)
                NAIP_Chips_folder = EIR.points_toNAIP_Chips(no_repro, inputName,'no')
            
            except:
                print ("No (no) NAIP imagery found")
                pass
            

    n_counter += 1
print ("All done!")
warnings.resetwarnings()  ## optional. I'm getting an number of depreciated warnings... 
